# 8.1 Variables

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The variables of a linear program have much in common with its numerical parameters.
Both are symbols that stand for numbers, and that may be used in arithmetic expressions.
Parameter values are supplied by the modeler or computed from other values, while the
values of variables are determined by an optimizing algorithm (as implemented in one of
the packages that we refer to as *solvers*).

Syntactically, variable declarations are the same as the parameter declarations defined
in [Chapter 7](../07/07.md), except that they begin with the keyword `var` rather than
`param`. The meaning of qualifying phrases within the declaration may be different,
however, when these phrases are applied to variables rather than to parameters.

Phrases beginning with `>=` or `<=` are by far the most common in declarations of variables
for linear programs. They have appeared in all of our examples, beginning with the
production model of [Figure 1-4](../01/tut_1_4.ipynb#fig-1-4a):

```ampl
var Make {p in PROD} >= 0, <= market[p];
```

This declaration creates an indexed collection of variables `Make[p]`, one for each member
`p` of the set `PROD`; the rules in this respect are exactly the same as for parameters.
The effect of the two qualifying phrases is to impose a restriction, or *constraint*, on the
permissible values of the variables. Specifically, `>= 0` implies that all of the variables
`Make[p]` must be assigned nonnegative values by the optimizing algorithm, while the
phrase `<= market[p]` says that, for each product `p`, the value given to `Make[p]` may
not exceed the value of the parameter `market[p]`.

In general, either `>=` or `<=` may be followed by any arithmetic expression involving
previously defined sets and parameters and currently defined dummy indices. Most linear
programs are formulated in such a way that every variable must be nonnegative; an AMPL
variable declaration can specify nonnegativity either directly by `>= 0`, or indirectly as
in the diet model of [Figure 5-1](../05/5_3_set_operations.ipynb#fig-5-1):

```ampl
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

var Buy {j in FOOD} >= f_min[j], <= f_max[j];
```

The values following `>=` and `<=` are lower and upper bounds on the variables. Because
these bounds represent a kind of constraint, they could just as well be imposed by the
constraint declarations described later in this chapter. By placing bounds in the `var`
declaration instead, you may be able to make the model shorter or clearer, although you
will not make the optimal solution any different or easier to find. Some solvers do treat
bounds specially in order to speed up their algorithms, but with AMPL all bounds are
identified automatically, no matter how they are expressed in your model.

Variable declarations may not use the comparison operators `<`, `>` or `<>` in qualifying
phrases. For linear programming it makes no sense to constrain a variable to be, say,
`< 3`, since it could always be chosen as `2.99999…` or as close to `3` as you like.

An `=` phrase in a variable declaration gives rise to a definition, as in a parameter
declaration. Because a variable is being declared, however, the expression to the right of
the `=` operator may contain previously declared variables as well as sets and parameters.
For example, instead of writing the complicated objective from the multi-period
production model of [Figure 6-3](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-3)
(`steelT3.mod`) as

```ampl
maximize Total_Profit:
  sum {p in PROD, t in 1..T}
     (sum {a in AREA[p]} revenue[p,a,t]*Sell[p,a,t]
      - prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
```

you could instead define variables to represent the total revenues, production costs, and
inventory costs:

```ampl
var Total_Revenue =
  sum {p in PROD, t in 1..T}
     sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t];

var Total_Prod_Cost =
  sum {p in PROD, t in 1..T} prodcost[p] * Make[p,t];

var Total_Inv_Cost =
  sum {p in PROD, t in 1..T} invcost[p] * Inv[p,t];
```

The objective would then be the sum of these three defined variables:

```ampl
maximize Total_Profit:
  Total_Revenue - Total_Prod_Cost - Total_Inv_Cost;
```

The structure of the objective is clearer this way. Also, the defined variables are
conveniently available to a `display` statement to show how the three main components of
profit compare:

```
ampl.display("Total_Revenue, Total_Prod_Cost, Total_Inv_Cost")

# Output:
# Total_Revenue = 801385
# Total_Prod_Cost = 285643
# Total_Inv_Cost = 1221
```

Declarations of defined variables like these do not give rise to additional constraints in
the resulting problem instance. Rather, the linear expression to the right of the `=` is
substituted for every occurrence of the defined variable in the objective and constraints.
Defined variables are even more useful for nonlinear programming, where the substitution
may be only implicit, so we will return to this topic in Chapter 18 xTODO.

If the expression to the right of the `=` operator contains no variables, then you are
merely defining variables to be fixed to values given by the data. In that case you should
use a `param` declaration instead. On the other hand, if you only want to fix some
variables temporarily while developing or analyzing a model, then you should leave the
declarations unchanged and instead fix them with the `fix` command described in
[Section 11.4] xTODO.

A `:=` or `default` phrase in a variable declaration gives initial values to the indicated
variables. Variables not assigned an initial value by `:=` can also be assigned initial
values from a data file. Initial values of variables are normally changed—ideally to
optimal values—when a solver is invoked. Thus the main purpose of initial values of
variables is to give the solver a good starting solution. Solvers for linear programming can
seldom make good use of a starting solution, however, so we defer further discussion of
this topic to Chapter 18 on nonlinear programming.

Finally, variables may be declared as `integer` so that they must take whole-number
values in any optimal solution, or as `binary` so that they may only take the values `0`
and `1`. Models that contain any such variables are *integer programs*, which are the
topic of Chapter 20 xTODO.
